In [1]:
import os
import pandas as pd
import numpy as np
import re
from __future__ import division
import math
from scipy.spatial import ConvexHull
import scipy
from configparser import ConfigParser
import glob
# from simba.rw_dfs import *

In [14]:
in_dir = '../Datasets/Original'
filesFound = glob.glob(in_dir + '/*.csv')
print('Extracting features from ' + str(len(filesFound)) + ' files...')
for currentFile in filesFound:
    print(currentFile)
    csv_df = pd.read_csv(currentFile)
    break
column_headers = csv_df.iloc[:,1:-2].columns.tolist()
csv_df

Extracting features from 98 files...
../Datasets/Original\11.4_Original_data.csv


,frame_number,"('nose', 'x')","('nose', 'y')","('nose', 'likelihood')","('H1R', 'x')","('H1R', 'y')","('H1R', 'likelihood')","('H2R', 'x')","('H2R', 'y')","('H2R', 'likelihood')",...,"('tail', 'y')","('tail', 'likelihood')","('S2', 'x')","('S2', 'y')","('S2', 'likelihood')","('S1', 'x')","('S1', 'y')","('S1', 'likelihood')",mouse_no,Freezing_label
0,0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,...,219.079468,0.998689,247.088928,281.114136,0.994912,292.150818,318.674744,0.995648,11.4,0
1,1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,...,219.292374,0.999059,247.683929,281.190613,0.996069,295.781158,321.270813,0.993003,11.4,0
2,2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,...,220.684891,0.999419,250.365112,281.190613,0.986211,299.371765,323.419800,0.987800,11.4,0
3,3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,...,222.606628,0.999285,253.343109,281.232605,0.981083,303.137299,326.057861,0.994152,11.4,0
4,4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,...,227.533890,0.999156,253.343109,281.232605,0.983985,304.960480,326.125854,0.995609,11.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,...,109.120758,0.997492,191.435760,106.898674,0.979662,230.204132,108.960442,0.977986,11.4,0
11246,11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,...,109.938370,0.999370,188.565491,107.834068,0.977227,230.204132,109.485367,0.980468,11.4,0
11247,11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,...,110.734329,0.999556,188.239838,111.964462,0.975709,230.204132,110.523743,0.985897,11.4,0
11248,11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,...,110.734329,0.999567,187.286819,111.964462,0.974826,226.513855,110.523743,0.992293,11.4,0


In [25]:
csv_df = csv_df.iloc[:,1:-2].copy()

In [26]:
columnHeaders = ["Mouse_nose_x", "Mouse_nose_y", "Mouse_nose_p", \
                         "Mouse_H1R_x", "Mouse_H1R_y", "Mouse_H1R_p", \
                         "Mouse_H2R_x", "Mouse_H2R_y", "Mouse_H2R_p", \
                         "Mouse_H1L_x", "Mouse_H1L_y", "Mouse_H1L_p", \
                         "Mouse_H2L_x", "Mouse_H2L_y", "Mouse_H2L_p", \
                         "Mouse_B1R_x", "Mouse_B1R_y", "Mouse_B1R_p", \
                         "Mouse_B2R_x", "Mouse_B2R_y", "Mouse_B2R_p", \
                         "Mouse_B3R_x", "Mouse_B3R_y", "Mouse_B3R_p", \
                         "Mouse_B1L_x", "Mouse_B1L_y", "Mouse_B1L_p", \
                         "Mouse_B2L_x", "Mouse_B2L_y", "Mouse_B2L_p", \
                         "Mouse_B3L_x", "Mouse_B3L_y", "Mouse_B3L_p", \
                         "Mouse_tail_x", "Mouse_tail_y", "Mouse_tail_p", \
                         "Mouse_S2_x", "Mouse_S2_y", "Mouse_S2_p", \
                         "Mouse_S1_x", "Mouse_S1_y", "Mouse_S1_p"]

csv_df.columns = columnHeaders

In [28]:
print('Evaluating convex hulls...')
########### MOUSE AREAS ###########################################
csv_df['Mouse_poly_area'] = csv_df.apply(lambda x: ConvexHull(np.array(
    [[x['Mouse_nose_x'], x["Mouse_nose_y"]],
     [x['Mouse_H1R_x'], x["Mouse_H1R_y"]],
     [x['Mouse_H2R_x'], x["Mouse_H2R_y"]],
     [x['Mouse_H1L_x'], x["Mouse_H1L_y"]], 
     [x['Mouse_H2L_x'], x["Mouse_H2L_y"]],
     [x['Mouse_B1R_x'], x["Mouse_B1R_y"]],
     [x['Mouse_B2R_x'], x["Mouse_B2R_y"]],
     [x['Mouse_B3R_x'], x["Mouse_B3R_y"]],
     [x['Mouse_B1L_x'], x["Mouse_B1L_y"]],
     [x['Mouse_B2L_x'], x["Mouse_B2L_y"]],
     [x['Mouse_B3L_x'], x["Mouse_B3L_y"]],
     [x['Mouse_tail_x'], x["Mouse_tail_y"]],
     [x['Mouse_S2_x'], x["Mouse_S2_y"]],
     [x['Mouse_S1_x'], x["Mouse_S1_y"]]])).area, axis=1)
csv_df

Evaluating convex hulls...


,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Mouse_tail_x,Mouse_tail_y,Mouse_tail_p,Mouse_S2_x,Mouse_S2_y,Mouse_S2_p,Mouse_S1_x,Mouse_S1_y,Mouse_S1_p,Mouse_poly_area
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,197.399582,219.079468,0.998689,247.088928,281.114136,0.994912,292.150818,318.674744,0.995648,599.294363
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,199.069702,219.292374,0.999059,247.683929,281.190613,0.996069,295.781158,321.270813,0.993003,602.073173
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,199.867844,220.684891,0.999419,250.365112,281.190613,0.986211,299.371765,323.419800,0.987800,603.542359
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,201.723312,222.606628,0.999285,253.343109,281.232605,0.981083,303.137299,326.057861,0.994152,607.245061
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,201.723312,227.533890,0.999156,253.343109,281.232605,0.983985,304.960480,326.125854,0.995609,623.814152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,115.261879,109.120758,0.997492,191.435760,106.898674,0.979662,230.204132,108.960442,0.977986,573.924363
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,115.261879,109.938370,0.999370,188.565491,107.834068,0.977227,230.204132,109.485367,0.980468,586.139715
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,115.261879,110.734329,0.999556,188.239838,111.964462,0.975709,230.204132,110.523743,0.985897,588.099724
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,115.261879,110.734329,0.999567,187.286819,111.964462,0.974826,226.513855,110.523743,0.992293,587.833687


In [30]:
########### CREATE SHIFTED DATAFRAME FOR DISTANCE CALCULATIONS ###########################################
csv_df_shifted = csv_df.shift(periods=1)
csv_df_shifted = csv_df_shifted.rename(
    columns = {
    "Mouse_nose_x": "Mouse_nose_x_shifted", "Mouse_nose_y": "Mouse_nose_y_shifted", "Mouse_nose_p": "Mouse_nose_p_shifted",
    "Mouse_H1R_x": "Mouse_H1R_x_shifted", "Mouse_H1R_y": "Mouse_H1R_y_shifted", "Mouse_H1R_p": "Mouse_H1R_p_shifted",
    "Mouse_H2R_x": "Mouse_H2R_x_shifted", "Mouse_H2R_y": "Mouse_H2R_y_shifted", "Mouse_H2R_p": "Mouse_H2R_p_shifted",
    "Mouse_H1L_x": "Mouse_H1L_x_shifted", "Mouse_H1L_y": "Mouse_H1L_y_shifted", "Mouse_H1L_p": "Mouse_H1L_p_shifted",
    "Mouse_H2L_x": "Mouse_H2L_x_shifted", "Mouse_H2L_y": "Mouse_H2L_y_shifted", "Mouse_H2L_p": "Mouse_H2L_p_shifted",
    "Mouse_B1R_x": "Mouse_B1R_x_shifted", "Mouse_B1R_y": "Mouse_B1R_y_shifted", "Mouse_B1R_p": "Mouse_B1R_p_shifted",
    "Mouse_B2R_x": "Mouse_B2R_x_shifted", "Mouse_B2R_y": "Mouse_B2R_y_shifted", "Mouse_B2R_p": "Mouse_B2R_p_shifted",
    "Mouse_B3R_x": "Mouse_B3R_x_shifted", "Mouse_B3R_y": "Mouse_B3R_y_shifted", "Mouse_B3R_p": "Mouse_B3R_p_shifted",
    "Mouse_B1L_x": "Mouse_B1L_x_shifted", "Mouse_B1L_y": "Mouse_B1L_y_shifted", "Mouse_B1L_p": "Mouse_B1L_p_shifted",
    "Mouse_B2L_x": "Mouse_B2L_x_shifted", "Mouse_B2L_y": "Mouse_B2L_y_shifted", "Mouse_B2L_p": "Mouse_B2L_p_shifted",
    "Mouse_B3L_x": "Mouse_B3L_x_shifted", "Mouse_B3L_y": "Mouse_B3L_y_shifted", "Mouse_B3L_p": "Mouse_B3L_p_shifted",
    "Mouse_tail_x": "Mouse_tail_x_shifted", "Mouse_tail_y": "Mouse_tail_y_shifted", "Mouse_tail_p": "Mouse_tail_p_shifted",
    "Mouse_S2_x": "Mouse_S2_x_shifted", "Mouse_S2_y": "Mouse_S2_y_shifted", "Mouse_S2_p": "Mouse_S2_p_shifted",
    "Mouse_S1_x": "Mouse_S1_x_shifted", "Mouse_S1_y": "Mouse_S1_y_shifted", "Mouse_S1_p": "Mouse_S1_p_shifted"
})

csv_df_combined = pd.concat([csv_df, csv_df_shifted], axis=1, join='inner')
csv_df_combined = csv_df_combined.fillna(0)
csv_df_combined = csv_df_combined.reset_index(drop=True)
csv_df_combined

,Mouse_nose_x,Mouse_nose_y,Mouse_nose_p,Mouse_H1R_x,Mouse_H1R_y,Mouse_H1R_p,Mouse_H2R_x,Mouse_H2R_y,Mouse_H2R_p,Mouse_H1L_x,...,Mouse_tail_x_shifted,Mouse_tail_y_shifted,Mouse_tail_p_shifted,Mouse_S2_x_shifted,Mouse_S2_y_shifted,Mouse_S2_p_shifted,Mouse_S1_x_shifted,Mouse_S1_y_shifted,Mouse_S1_p_shifted,Mouse_poly_area
0,389.652130,391.121582,0.999969,368.418610,395.140015,0.999760,346.555359,395.608948,0.996968,391.455475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,390.096405,393.142700,0.999950,368.954803,397.121368,0.999568,351.003540,399.964783,0.997916,391.991150,...,197.399582,219.079468,0.998689,247.088928,281.114136,0.994912,292.150818,318.674744,0.995648,599.294363
2,390.536285,397.035248,0.999945,369.900299,400.646271,0.999638,352.932281,400.691162,0.999153,392.961304,...,199.069702,219.292374,0.999059,247.683929,281.190613,0.996069,295.781158,321.270813,0.993003,602.073173
3,398.239349,397.035248,0.999765,375.819672,400.646271,0.999534,357.683594,400.691162,0.999468,399.517914,...,199.867844,220.684891,0.999419,250.365112,281.190613,0.986211,299.371765,323.419800,0.987800,603.542359
4,415.408661,397.035248,0.999855,396.818848,400.646271,0.999252,374.967102,400.691162,0.998945,413.368073,...,201.723312,222.606628,0.999285,253.343109,281.232605,0.981083,303.137299,326.057861,0.994152,607.245061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,364.107910,149.601181,0.998303,344.420105,150.693054,0.999295,323.575958,153.616699,0.992387,361.650970,...,115.967751,108.480583,0.996206,191.435760,106.086227,0.988119,236.414139,105.547989,0.990182,569.459741
11246,375.265228,128.008606,0.997676,351.077271,137.533630,0.993547,324.854950,142.170212,0.986537,365.810822,...,115.261879,109.120758,0.997492,191.435760,106.898674,0.979662,230.204132,108.960442,0.977986,573.924363
11247,377.197815,117.285118,0.999162,355.023926,131.847992,0.986025,325.591003,140.211060,0.933739,367.412567,...,115.261879,109.938370,0.999370,188.565491,107.834068,0.977227,230.204132,109.485367,0.980468,586.139715
11248,377.197815,113.858429,0.999720,355.023926,122.189278,0.990261,325.591003,135.173889,0.938426,367.412567,...,115.261879,110.734329,0.999556,188.239838,111.964462,0.975709,230.204132,110.523743,0.985897,588.099724


In [ ]:
print('Calculating euclidean distances...')
########### EUCLIDEAN DISTANCES ###########################################
csv_df['Nose_to_tail'] = (np.sqrt((csv_df.Mouse_nose_x - csv_df.Mouse_tail_x) ** 2 + (csv_df.Mouse_nose_y - csv_df.Mouse_tail_y) ** 2))
csv_df['Distance_feet'] = (np.sqrt((csv_df.Mouse1_left_foot_x - csv_df.Mouse1_right_foot_x) ** 2 + (csv_df.Mouse1_left_foot_y - csv_df.Mouse1_right_foot_y) ** 2))
csv_df['Distance_hands'] = (np.sqrt((csv_df.Mouse1_left_hand_x - csv_df.Mouse1_right_hand_x) ** 2 + (csv_df.Mouse1_left_hand_y - csv_df.Mouse1_right_hand_y) ** 2))
csv_df['Distance_ears'] = (np.sqrt((csv_df.Mouse1_left_ear_x - csv_df.Mouse1_right_ear_x) ** 2 + (csv_df.Mouse1_left_ear_y - csv_df.Mouse1_right_ear_y) ** 2))
csv_df['Distance_unilateral_left_hands_feet'] = (np.sqrt((csv_df.Mouse1_left_foot_x - csv_df.Mouse1_left_hand_x) ** 2 + (csv_df.Mouse1_left_foot_y - csv_df.Mouse1_left_hand_y) ** 2))
csv_df['Distance_unilateral_right_hands_feet'] = (np.sqrt((csv_df.Mouse1_right_foot_x - csv_df.Mouse1_right_hand_x) ** 2 + (csv_df.Mouse1_right_foot_y - csv_df.Mouse1_right_hand_y) ** 2))
csv_df['Distance_bilateral_left_foot_right_hand'] = (np.sqrt((csv_df.Mouse1_left_foot_x - csv_df.Mouse1_right_hand_x) ** 2 + (csv_df.Mouse1_left_foot_y - csv_df.Mouse1_right_hand_y) ** 2))
csv_df['Distance_bilateral_right_foot_left_hand'] = (np.sqrt((csv_df.Mouse1_right_foot_x - csv_df.Mouse1_left_hand_x) ** 2 + (csv_df.Mouse1_right_foot_y - csv_df.Mouse1_left_hand_y) ** 2))
csv_df['Distance_back_tail'] = (np.sqrt((csv_df.Mouse1_back_x - csv_df.Mouse1_tail_x) ** 2 + (csv_df.Mouse1_back_y - csv_df.Mouse1_tail_y) ** 2))
csv_df['Distance_back_nose'] = (np.sqrt((csv_df.Mouse1_back_x - csv_df.Mouse1_nose_x) ** 2 + (csv_df.Mouse1_back_y - csv_df.Mouse1_nose_y) ** 2))

csv_df['Movement_nose'] = (np.sqrt((csv_df_combined.Mouse1_nose_x_shifted - csv_df_combined.Mouse1_nose_x) ** 2 + (csv_df_combined.Mouse1_nose_y_shifted - csv_df_combined.Mouse1_nose_y) ** 2))
csv_df['Movement_back'] = (np.sqrt((csv_df_combined.Mouse1_back_x_shifted - csv_df_combined.Mouse1_back_x) ** 2 + (csv_df_combined.Mouse1_back_y_shifted - csv_df_combined.Mouse1_back_y) ** 2))
csv_df['Movement_left_ear'] = (np.sqrt((csv_df_combined.Mouse1_left_ear_x_shifted - csv_df_combined.Mouse1_left_ear_x) ** 2 + (csv_df_combined.Mouse1_left_ear_y_shifted - csv_df_combined.Mouse1_left_ear_y) ** 2))
csv_df['Movement_right_ear'] = (np.sqrt((csv_df_combined.Mouse1_right_ear_x_shifted - csv_df_combined.Mouse1_right_ear_x) ** 2 + (csv_df_combined.Mouse1_right_ear_y_shifted - csv_df_combined.Mouse1_right_ear_y) ** 2))
csv_df['Movement_left_foot'] = (np.sqrt((csv_df_combined.Mouse1_left_foot_x_shifted - csv_df_combined.Mouse1_left_foot_x) ** 2 + (csv_df_combined.Mouse1_left_foot_y_shifted - csv_df_combined.Mouse1_left_foot_y) ** 2))
csv_df['Movement_right_foot'] = (np.sqrt((csv_df_combined.Mouse1_right_foot_x_shifted - csv_df_combined.Mouse1_right_foot_x) ** 2 + (csv_df_combined.Mouse1_right_foot_y_shifted - csv_df_combined.Mouse1_right_foot_y) ** 2))
csv_df['Movement_tail'] = (np.sqrt((csv_df_combined.Mouse1_tail_x_shifted - csv_df_combined.Mouse1_tail_x) ** 2 + (csv_df_combined.Mouse1_tail_y_shifted - csv_df_combined.Mouse1_tail_y) ** 2))
csv_df['Movement_left_hand'] = (np.sqrt((csv_df_combined.Mouse1_left_hand_x_shifted - csv_df_combined.Mouse1_left_hand_x) ** 2 + (csv_df_combined.Mouse1_left_hand_y_shifted - csv_df_combined.Mouse1_left_hand_y) ** 2))
csv_df['Movement_right_hand'] = (np.sqrt((csv_df_combined.Mouse1_right_hand_x_shifted - csv_df_combined.Mouse1_right_hand_x) ** 2 + (csv_df_combined.Mouse1_right_hand_y_shifted - csv_df_combined.Mouse1_right_hand_y) ** 2))
csv_df['Mouse_polygon_size_change'] = (csv_df_combined['Mouse_poly_area_shifted']) - (csv_df_combined['Mouse_poly_area'])